# Contexto ✒️

Já familiarizado com a arquitetura da rede e a sua implementação, bem como a estrutura de dados pertinente para o procedimento do aprendizado da máquina no contexto das redes neurais, esse notebook irá debruçar sobre a etapa de forward da rede neural até a função objetivo.

# Bibliotecas 📚

In [203]:
import torch
import pandas as pd

from torch import nn
from torch import optim
from torchsummary import summary

from sklearn import datasets
from sklearn.preprocessing import StandardScaler

### Função de perda

A função de perda pode ser pensada como uma medida numérica de qualidade do modelo, a qual se relaciona numa busca por maximização ou minimização de algo.

- Exemplo de maximização :

Modelando o desempenho de um carro de fórmula 1 numa função matemática, a função de perda se debruçaria sobre as features que podem prover a maximização do veículo, para que provesse a ele um incremento de performance.

- Exemplo de minimazação :    

Compreendendo a elaboração de preços de casas com base nas features que se colocam como características latentes a esse, eu poderia criar um modelo que prevesse as casas com base em suas características, local em que está situada e etc. A prova de qualidade desse modelo seria a sua previsão estar próxima ou de modo colinear ao valor da casa real. Ou seja, a sua função de perda se debruçaria em minimizar a distância do que é predito em relação ao o que é encontrado. Podemos encontrar a função de perda, ainda, com outros nomes, como função de custo, loss, função objetivo e etc.

 A escolha da função de perda a ser utilizada está intimamente relacionada ao problema no qual ela se circunscreve. Tomando como exemplo problemas de regressão e classificação, as funções de perda comumente utilizadas para cada qual são :

 >

 ### Regressão

 Erro absoluto médio (l1) - $[y -y'i]$

 Erro quadrático médio (MSE / l2) - $[y -y'i]²$

 >

 ### Classificação

 Normalmente utiliza-se a entropia cruzada, que mensura a diferença do valor inferido em relação ao valor encontrado (ou real).  

## Para problemas de classificação 🥇🥈🥉

Para auxiliar no processo da implementação, o dataset escolhido se refere a vinhos, os quais relaciona as suas características com a classe que pertencem.

In [218]:
# Carregando o dataset:

wine = datasets.load_wine()

# Features
data_wine = wine.data

# Target
target_wine = wine.target

# Formato das features e da target.
print(data_wine.shape)
print(target_wine.shape)

(178, 13)
(178,)


In [205]:
# Nome das features referentes ao vinho.

print(wine.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


## Criando a rede neural para classificação ♦️



In [219]:
input_size = data_wine.shape[1]
hidden_size = 32
output_size = len(wine.target_names)

In [217]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):

    # Instanciando nn.Module
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.Softmax()



  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output



In [220]:
net_wine = WineClassifier(input_size, hidden_size, output_size) # Quando terminar tudo acrescentar : .to(device) para fazer o cast na GPU.

In [221]:
# Visualizando a rede criada.

net_wine

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)

In [222]:
'''
Definindo o critério da função de objetivo. Como se trata
de um problema de classificação, estou utilizando a entropria cruzada.
'''

criterion_wine = nn.CrossEntropyLoss() # fazer depois o cast na GPU.


Uma vez criado a rede neural e a definição da métrica para a função objetivo, precisa-se transformar os dados para que fiquem no tipo tensor, caso não estejam. Tendo em vista que os dados presentes estão na forma de array, precisa-se transformá-los em tensor.

In [223]:
# Transformando os dados em tensores.

X_tensor = torch.from_numpy(data_wine).float()
y_tensor = torch.from_numpy(target_wine)

'''
Fazer o cast na GPU depois para os tensores criados.

X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)

'''

print(X_tensor.dtype, y_tensor.dtype)

torch.float32 torch.int64


In [224]:
# Realizando o forward na rede.

pred = net_wine(X_tensor)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [225]:
print(f'Visualizando o formato da predição e do rótulo \n')
print(f'Predição : {pred.shape}')
print(f'Rótulo : {y_tensor.shape}')

Visualizando o formato da predição e do rótulo 

Predição : torch.Size([178, 3])
Rótulo : torch.Size([178])


Por mais que o formato entre o forward e o rótulo seja diferente, apresentando portanto diferentes dimensões, não há problema, para cenários de classificação, essa dissonância, sendo até esperada, o que não exige nenhum grau de tratamento e etc.

In [226]:
'''
O valor encontrado pela loss, na forma em que está, retorna a média de perda
entre o valor real e o inferido pela rede, com base na entropia cruzada,
informando a qualidade da rede criada.
'''

loss = criterion_wine(pred, y_tensor)
loss

tensor(1.1644, grad_fn=<NllLossBackward0>)

In [227]:
# Se eu quisésse encontrar para um valor ou range.

# Para um valor

loss = criterion_wine(pred[0], y_tensor[0])
loss



tensor(1.5514, grad_fn=<NllLossBackward0>)

In [228]:
# Para um alcance

loss = criterion_wine(pred[0:10], y_tensor[0:10])
loss

tensor(1.5513, grad_fn=<NllLossBackward0>)

## Para problemas de regressão ♣️

In [229]:
# Carregando o dataset utilizado

diabetes = datasets.load_diabetes()

# Feature
data = diabetes.data

# Target
target = diabetes.target

In [230]:
# Visualizando as features referentes à target.

diabetes.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [231]:
print(data.shape)
print(target.shape)

(442, 10)
(442,)


In [232]:
# Parâmetros a serem passados atrelados ao problema de regressão.

input_size = data.shape[1]
hidden_size = 32
output_size = 1

In [233]:
# Implementando a rede para regressão :

class DiabetesRegressor(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):

    # Instanciando nn.Module
    super(DiabetesRegressor, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.sigmoid(self.out(feature))

    return output



Ao analisar a arquitetura da rede, observa-se que não estou utilizando a função de ativação, na porção de saída, softmax, mas, sim, uma linear, pois se trata de um caso de regressão no qual deseja-se prever um valor com base nas features.

In [234]:
net = DiabetesRegressor(input_size, hidden_size, output_size) # Não esquecer de depois fazer o cast na GPU.

In [235]:
# Visualizando a rede neural construída.

print(net)

DiabetesRegressor(
  (hidden): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [236]:
# Criando a função de perda.

criterion = nn.MSELoss() # Passar posteriormente à GPU

In [237]:
# Não esquecer de depois passar para a GPU

X_tensor = torch.from_numpy(data).float()
y_tensor = torch.from_numpy(target)

In [238]:
print(X_tensor.shape, y_tensor.shape)

torch.Size([442, 10]) torch.Size([442])


In [239]:
# Fazendo o forward na rede

pred = net(X_tensor)

In [240]:
pred.shape

torch.Size([442, 1])

In [241]:
'''
Para calcular a loss, basta relacionar os valores do forward
em relação aos esperados. Porém, diferente do cálculo anterior
no qual podia-se passar o foward da rede com formato diferente
ao encontrado nos valores esperados, para regressões o valor tem
de ser os mesmos, ou seja, tanto o forward quanto o valor esperado
devem estar na mesma dimensão. Para isso, basta :
'''

loss = criterion(pred.squeeze(), y_tensor)
loss



tensor(28917.9163, dtype=torch.float64, grad_fn=<MseLossBackward0>)

Novamente, o valor encontrado pela loss se refere a média
da diferença do valor real entre o esperado, com base na
métrica utilizada.
